In [19]:
!pip install python-chess~=0.26







In [20]:
!pip install livelossplot==0.3.4


In [21]:
!wget https://www.dropbox.com/sh/75gzfgu7qo94pvh/AACk_w5M94GTwwhSItCqsemoa/Stockfish%205/stockfish-5-linux.zip


--2020-04-15 14:02:05--  https://www.dropbox.com/sh/75gzfgu7qo94pvh/AACk_w5M94GTwwhSItCqsemoa/Stockfish%205/stockfish-5-linux.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/75gzfgu7qo94pvh/AACk_w5M94GTwwhSItCqsemoa/Stockfish%205/stockfish-5-linux.zip [following]
--2020-04-15 14:02:05--  https://www.dropbox.com/sh/raw/75gzfgu7qo94pvh/AACk_w5M94GTwwhSItCqsemoa/Stockfish%205/stockfish-5-linux.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc44c85ad1e617ce657976f47a31.dl.dropboxusercontent.com/cd/0/inline/A16Pxiro0WGHXGchNWpdqJ-POqlNQFq8czCUWKHrD7qLXsuhl6SBMmop6OgbOTsikLC_a_FRsMmL2JRHIB-IavSvf2N3vme9Ky18dFvL577rXQ/file# [following]
--2020-04-15 14:02:06--  https://uc44c85ad1e617ce657976f47a31.dl.dropboxusercontent.c

In [22]:
!unzip stockfish-5-linux.zip


Archive:  stockfish-5-linux.zip
replace stockfish-5-linux/Readme.md? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: stockfish-5-linux/Readme.md  
replace stockfish-5-linux/polyglot.ini? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace stockfish-5-linux/polyglot.ini? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: stockfish-5-linux/polyglot.ini  
  inflating: stockfish-5-linux/logo.bmp  
  inflating: stockfish-5-linux/Copying.txt  
  inflating: stockfish-5-linux/src_c++11/ucioption.h  
  inflating: stockfish-5-linux/src_c++11/ucioption.cpp  
  inflating: stockfish-5-linux/src_c++11/uci.cpp  
  inflating: stockfish-5-linux/src_c++11/types.h  
  inflating: stockfish-5-linux/src_c++11/tt.h  
  inflating: stockfish-5-linux/src_c++11/tt.cpp  
  inflating: stockfish-5-linux/src_c++11/timeman.h  
  inflating: stockfish-5-linux/src_c++11/timeman.cpp  
  inflating: stockfish-5-linux/src_c++11/thread.h  
  inflating: stockfish-5-linux/src_c++11/thread.cpp  


In [0]:
!chmod +x stockfish-5-linux/Linux/stockfish_14053109_x64


In [0]:
import torch
import pickle

import chess
import chess.pgn
import chess.engine

from tqdm import tqdm

from AIChess import evaluate
from DeepChess import boardToTensor

STOCKFISH_PATH = "stockfish"
STOCKFISH_PATH = ".\stockfish-5-linux/Linux/stockfish_14053109_x64"

# Nouvelle section

In [0]:

def loadData():
    """
    Loads the data from a pgn file
    """

    filePath = "ficsgamesdb_201901_CvC_nomovetimes_120511.pgn" # smaller dataset !!
    #filePath = "ficsgamesdb_2019_chess_nomovetimes_124014.pgn" # dataset of all games from FICS in 2019 !!

    with open(filePath) as pgn:
        nbGames = len(pgn.readlines()) // 22 # nb de lignes que prend un jeu (header + moves + blancs autour de moves) !!

        print("{} games found\n".format(nbGames))

        pgn.seek(0)

        games = []
        nbStates = 0

        game = chess.pgn.read_game(pgn)

        for i in tqdm(range(nbGames // 100), desc="Parsing games", unit="game"):
            games.append(game)

            nbStates += int(game.headers['PlyCount'])

            game = chess.pgn.read_game(pgn)

    print("Completed. {} games have been parsed\n".format(len(games)))

    X = []
    y = []

    engine = chess.engine.SimpleEngine.popen_uci(STOCKFISH_PATH)

    for game in tqdm(games, desc="Generating states", unit="game"):

        node = game.end()

        while node.parent is not None:
            position = node.board()
            tensor = boardToTensor(position)

            output = engine.analyse(position, chess.engine.Limit(time=0.001, depth =  0))['score']

            # output = evaluate(position) # stockfish to plug here

            X.append(tensor)
            y.append(output)

            node = node.parent

    print("Completed. {} states have been generated\n".format(len(X)))

    return X, y

def save(obj, filePath):
    with open(filePath, "wb") as file:
        pickle.dump(obj, file)

def load(filePath):
    with open(filePath, "rb") as file:
        return pickle.load(file)

In [35]:

if __name__ == "__main__":

    print("############################################################")
    print("#################### GENERATING DATASET ####################")
    print("############################################################\n")

    X, y = loadData()

    print("############################################################")
    print("#################### SAVING DATASET ########################")
    print("############################################################\n")
    
    save(X, "chessInput")
    save(y, "chessOutput")

    print(len(load("chessOutput")))


Parsing games: 100%|██████████| 24/24 [00:00<00:00, 184.94game/s]

############################################################
#################### GENERATING DATASET ####################
############################################################

2495 games found

Completed. 24 games have been parsed



AttributeError: ignored